## PRINCIPAL COMPONENT ANALYSIS

Consider daily closing prices of the “FANG” stocks from 1/1/2019 until 12/1/2019

Facebook (FB)

Apple (AAPL)

Netflix (NFLX)

Google (GOOG)

(you can download the closing prices from Yahoo Finance)

+ Determine daily return of these stocks
+ Calculate the four Principal Components of these daily returns
+ Standardize the data and calculate the correlation matrix
+ Calculate the four Principal Components of the standardized daily returns

In [1]:
import pandas_datareader as pdr
import datetime

### Determine daily return of these stocks

In [11]:
stocks = ['FB', 'AAPL', 'NFLX', 'GOOG']
# stocks = 'FB'
start = datetime.datetime(2019,1, 1)
end = datetime.datetime(2019,12, 1)
flag = 0
for stock in stocks:
    if flag == 0:
        price_df = pdr.DataReader(stock, 'yahoo',start,end)
        price_df = price_df[['Adj Close']]
        
        flag = 1
    else:
        
        price_df = pd.merge(price_df, pdr.DataReader(stock, 'yahoo',start,end)[['Adj Close']], on='Date')
price_df.columns = stocks

In [13]:
daily_return = price_df.pct_change(1) 
daily_return.head()

,FB,AAPL,NFLX,GOOG
Date,,,,
2019-01-02,NaN,NaN,NaN,NaN
2019-01-03,-0.029039,-0.099607,0.013226,-0.028484
2019-01-04,0.047138,0.042689,0.097234,0.053786
2019-01-07,0.000725,-0.002226,0.059717,-0.002167
2019-01-08,0.032452,0.019063,0.015634,0.007385


### Calculate the four Principal Components of these daily returns